# Homework 5-2: Developing a code for carbonate systems
## Abby Baskind
## 5 October 2023

hi gabby

In [1]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

# Import K's code for calculating the coefficients of the carb system
import calc_coeffs as co2
import H_poly as hpoly
from importlib import reload
import warnings
# warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

In [5]:
def pHsolver(TA = None, temperature = None, salinity = None, pCO2 = None, pHlo = None, pHhi = None):
    import calc_coeffs as co2
    import H_poly as hpoly
    import numpy as np 
    import math
    
    """
    This function, as it is currently written, takes TA [umol/kg], temperature [degC], 
    salinity [PSU], and pCO2 [uatm] to solve for pH [total] and HCO3, CO3, CO2*, and DIC,
    all in [umol/kg].
    
    Arguments for TA, T, S, and pCO2 can be of any length as long as they are all the same
    length as each other and do not have NaN values. 
    
    Required dependencies:
        - calc_coeffs.py: calculates carbonate system coefficients
            function [k1,k2,k0,kb,kw,BT]=calc_coeffs(T,S)
            T   = temperature (degrees C)
            S   = salinity (PSU)
        - H_poly.py: H polynomial root finder
    
    Inputs:
        - total alkalinity in umol/kg
        - pCO2 in uatm
        - temperature in °C
        - salinity in PSU
        - OPTIONAL: pHlo and pHhi in total scale
            - if pHlo and pHhi not provided, default to 6 and 9
    
    Returns:
        - [H+] in umol/kg
        - pH in total scale
        - [HCO3] in umol/kg
        - [CO3] in umol/kg
        - [CO2*] in umol/kg
        - Csat = [HCO3] + [CO3] + [CO2*] in umol/kg
    """
    
    
    # Check that all the inputs are provided
    if np.asarray(TA).any() == None:
        raise KeyError('Please provide TA [umol/kg] with kwarg "TA = *value*".')
    if np.asarray(temperature).any() == None:
        raise KeyError('Please provide temperature [degC] with kwarg "temperature = *value*".')
    if np.asarray(salinity).any() == None:
        raise KeyError('Please provide salinity [PSU] with kwarg "salinity = *value*".')
    if np.asarray(pCO2).any() == None:
        raise KeyError('Please provide pCO2 [uatm] with kwarg "pCO2 = *value*".')
        
    # If args are not already numpy ndarrays,
    # Convert them to ndarrays
    if not isinstance(TA,np.ndarray):
        TA = np.asarray(TA)
    if not isinstance(temperature,np.ndarray):
        temperature = np.asarray(temperature)
    if not isinstance(salinity,np.ndarray):
        salinity = np.asarray(salinity)
    if not isinstance(pCO2,np.ndarray):
        pCO2 = np.asarray(pCO2)
        
    # Check for NaN values
    if np.isnan(np.min(TA)) or np.isnan(np.min(temperature)) or np.isnan(np.min(salinity)) or np.isnan(np.min(pCO2)):
        raise Exception("No NaN values allowed.") 
    
    # Check all input arrays are same size
    if not (TA.size == temperature.size and TA.size == salinity.size and TA.size == pCO2.size):
        raise Exception("Input arrays are not the same size.") 
    # If pH limits are not explicitly stated
    # Default to 6 and 9
    if pHlo == None or pHhi == None:
        pHhi = 9
        pHlo = 6
        
    # Check magnitude of TA and pCO2 to guess if units are right
    if not TA.min() > 1:
        raise Exception('This function takes TA units of umol/kg. Based on the magnitude of your input, we assume you entered TA in mol/kg.')
    if not pCO2.min() > 1:
        raise Exception('This function takes pCO2 units of uatm. Based on the magnitude of your input, we assume you entered pCO2 in atm.')
    TA = TA*1e-6
    pCO2 = pCO2*1e-6
    
    # Calculate the coefficients
    coeffs = co2.calc_coeffs(temperature, salinity)
    k0 = coeffs['k0']
    k1 = coeffs['k1']
    k2 = coeffs['k2']
    kb = coeffs['kb']
    kw = coeffs['kw']
    BT = coeffs['BT']
    
    # Calculate CO2* from pCO2
    # K0 = [CO2*]/pCO2
    co2star = k0 * pCO2
    
    # If args are not already numpy ndarrays,
    # Convert them to ndarrays
    if not isinstance(TA,np.ndarray):
        TA = np.asarray(TA)
        print(type(TA))
    if not isinstance(temperature,np.ndarray):
        temperature = np.asarray(temperature)
    if not isinstance(salinity,np.ndarray):
        salinity = np.asarray(salinity)
    if not isinstance(pCO2,np.ndarray):
        pCO2 = np.asarray(pCO2)
    
    # Initialize arrays to store results
    pH = np.zeros(TA.size)
    H = np.zeros(TA.size)
    HCO3 = np.zeros(TA.size)
    CO3 = np.zeros(TA.size)
    Csat = np.zeros(TA.size)
    
    # If inputs are size 1 are provided
    if TA.size == 1:
        Hhi = 10**(-pHlo)
        phlo = 6
        Hlo = 10**(-pHhi)
        Hmid = np.mean([Hhi,Hlo])
        threshold = Hhi-Hlo
        while np.abs(threshold) > 5e-23:
            Hmid = np.mean([Hhi,Hlo])
            sol = hpoly.H_poly(Hmid, TA, co2star, k0, k1, k2, kb, BT)
            if sol > 0:
                Hhi = Hmid
            elif sol < 0:
                Hlo = Hmid
            if Hlo > Hhi:
                print('mix up')
                break
            threshold = Hhi-Hlo
        Hmid = np.mean([Hhi,Hlo])
        H = Hmid
        pH = -math.log10(H)
    
        # K1 = [HCO3][H]/[CO2*]
        HCO3 = (k1 * co2star)/H
        # k2 = [CO3][H]/[HCO3]
        CO3 = (k2 * HCO3)/H
        # Csat = CO3 + CO2* + HCO3
        Csat = CO3 + co2star + HCO3 
    
    # If longer inputs provided
    else:
        for i in np.arange(0,TA.size)-1: 
            Hhi = 10**(-pHlo)
            phlo = 6
            Hlo = 10**(-pHhi)
            Hmid = np.mean([Hhi,Hlo])
            threshold = Hhi-Hlo
            while np.abs(threshold) > 5e-23:
                Hmid = np.mean([Hhi,Hlo])
                sol = hpoly.H_poly(Hmid, TA[i], co2star[i], k0[i], k1[i], k2[i], kb[i], BT[i])
                if sol > 0:
                    Hhi = Hmid
                elif sol < 0:
                    Hlo = Hmid
                if Hlo > Hhi:
                    print('mix up')
                    break
                threshold = Hhi-Hlo
            Hmid = np.mean([Hhi,Hlo])
            H[i] = Hmid
            pH[i] = -math.log10(H[i])
    
            # K1 = [HCO3][H]/[CO2*]
            HCO3[i] = (k1[i] * co2star[i])/H[i]
            # k2 = [CO3][H]/[HCO3]
            CO3[i] = (k2[i] * HCO3[i])/H[i]
            # Csat = CO3 + CO2* + HCO3
            Csat[i] = CO3[i] + co2star[i] + HCO3[i]
    
    # Return all the data in a dictionary 
    data = {
        '[H+]': H/1e-6,
        'pH': pH,
        '[HCO3]': HCO3/1e-6,
        '[CO3]': CO3/1e-6,
        '[CO2*]': co2star/1e-6,
        'Csat': Csat/1e-6,
        }
    return data
        

## Note that the inputs and outputs of my function are in umol/kg

In [6]:
results = pHsolver(TA = 2350, temperature = 20, salinity = 34.5, pCO2 = 380, pHlo = 7, pHhi = 9)
results

<class 'numpy.ndarray'>


{'[H+]': 0.008628886017489654,
 'pH': 8.064045267747677,
 '[HCO3]': 1871.5772430896734,
 '[CO3]': 197.68372333702308,
 '[CO2*]': 12.348181085475977,
 'Csat': 2081.6091475121725}

In [7]:
ta = np.asarray([2350,2000])
T = np.asarray([20,20])
S = np.asarray([30,34.5])
pCO2 = np.asarray([400,380])
pHsolver(TA = ta, temperature = T, salinity = S, pCO2 = pCO2)

{'[H+]': array([0.00867251, 0.00992581]),
 'pH': array([8.06185511, 8.00323418]),
 '[HCO3]': array([1920.58913364, 1627.03413461]),
 '[CO3]': array([180.49158635, 149.39935735]),
 '[CO2*]': array([13.31836347, 12.34818109]),
 'Csat': array([2114.39908346, 1788.78167304])}

In [8]:
ta = np.asarray([2350])
T = np.asarray([20,20])
S = np.asarray([30,34.5])
pCO2 = np.asarray([400,380])
pHsolver(TA = ta, temperature = T, salinity = S, pCO2 = pCO2)

Exception: Input arrays are not the same size.

In [9]:
ta = np.asarray([2350, 2000e-6])
T = np.asarray([20,20])
S = np.asarray([30,34.5])
pCO2 = np.asarray([400,380])
pHsolver(TA = ta, temperature = T, salinity = S, pCO2 = pCO2)

Exception: This function takes TA units of umol/kg. Based on the magnitude of your input, we assume you entered TA in mol/kg.

In [ ]:
ta = np.asarray([2350, np.n])
T = np.asarray([20,20])
S = np.asarray([30,34.5])
pCO2 = np.asarray([400,380])
pHsolver(TA = ta, temperature = T, salinity = S, pCO2 = pCO2)